Speaker : Chieh

# Outline
- Our First Convolutional Model
    - Convolutions
    - Pooling
    - Dropout
- History of CNN Architectures
    - AlexNet
    - Inception/GoogLeNet
    - VGG
    - ResNet
- Using Pretrained Models in PyTorch
    - Examining a Model’s Structure
    - BatchNorm
- PyTorch Hub

# Introduction


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

# Convolutional Neural Networks

```
nn.Conv2d(in_channels,out_channels, kernel_size, stride, padding)
```

- in_channels: the number of input channels that we’ll be receiving in the
layer.
- out_channels: the number of output channels, which corresponds to the number of filters in our conv layer.
- kernel_size: the height and width of our filter.
- stride: how many steps across the input we move when we adjust the filter to a new position.
- padding

```
nn.MaxPool2d(kernel_size, stride)
```
We take the maximum value from each of these tensors.

```
nn.AdaptiveMaxPool2d((output_size))
nn.AdaptiveAvgPool2d((output_size))
```
These work independently of the incoming input tensor’s dimensions

## Create a model

In [0]:
class CNNNet(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__()
        
        self.features = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2), 
        nn.ReLU(), 
        nn.MaxPool2d(kernel_size=3, stride=2),

        nn.Conv2d(64, 192, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        
        nn.Conv2d(192, 384, kernel_size=3, padding=1),
        nn.ReLU(),
        
        nn.Conv2d(384, 256, kernel_size=3, padding=1),
        nn.ReLU(),
        
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        self.classifier = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256 * 6 * 6, 4096),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Linear(4096, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## Call the model

In [0]:
cnnnet = CNNNet()

## Train the model

In [0]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

## Check the result

In [0]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])
train_data_path = "/content/drive/My Drive/Colab Notebooks/data/train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "/content/drive/My Drive/Colab Notebooks/data/val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
batch_size=64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [0]:
cnnnet.to(device)
optimizer = optim.Adam(cnnnet.parameters(), lr=0.001)

In [0]:
train(cnnnet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=10, device=device)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 0, Training Loss: 0.85, Validation Loss: 0.70, accuracy = 0.21
Epoch: 1, Training Loss: 0.68, Validation Loss: 0.79, accuracy = 0.68
Epoch: 2, Training Loss: 0.63, Validation Loss: 0.58, accuracy = 0.73
Epoch: 3, Training Loss: 0.53, Validation Loss: 0.39, accuracy = 0.81
Epoch: 4, Training Loss: 0.53, Validation Loss: 0.55, accuracy = 0.61
Epoch: 5, Training Loss: 0.49, Validation Loss: 0.57, accuracy = 0.70
Epoch: 6, Training Loss: 0.43, Validation Loss: 0.42, accuracy = 0.76
Epoch: 7, Training Loss: 0.39, Validation Loss: 0.58, accuracy = 0.71
Epoch: 8, Training Loss: 0.38, Validation Loss: 0.57, accuracy = 0.72
Epoch: 9, Training Loss: 0.37, Validation Loss: 0.78, accuracy = 0.63


# Using Pretrained Models in PyTorch

In [0]:
import torchvision.models as models
alexnet = models.alexnet(num_classes=2)

In [0]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [0]:
alexnet.to(device)
optimizer_alexnet = optim.Adam(alexnet.parameters(), lr=0.001)

In [0]:
train(alexnet, optimizer_alexnet, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=10, device=device)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 0, Training Loss: 1.00, Validation Loss: 0.70, accuracy = 0.21
Epoch: 1, Training Loss: 0.69, Validation Loss: 0.69, accuracy = 0.70
Epoch: 2, Training Loss: 0.66, Validation Loss: 0.64, accuracy = 0.62
Epoch: 3, Training Loss: 0.57, Validation Loss: 0.33, accuracy = 0.84
Epoch: 4, Training Loss: 0.63, Validation Loss: 0.66, accuracy = 0.66
Epoch: 5, Training Loss: 0.51, Validation Loss: 0.98, accuracy = 0.56
Epoch: 6, Training Loss: 0.54, Validation Loss: 0.50, accuracy = 0.69
Epoch: 7, Training Loss: 0.48, Validation Loss: 0.31, accuracy = 0.84
Epoch: 8, Training Loss: 0.42, Validation Loss: 0.42, accuracy = 0.83
Epoch: 9, Training Loss: 0.42, Validation Loss: 0.38, accuracy = 0.80


## Examining a Model’s Structure

In [0]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

# PyTorch Hub
One-Stop Shopping for Models: It is a central location for obtaining any published model.

In [18]:
resnet18 = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_master


In [19]:
print(resnet18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

We can discover all the models inside that repository that are available to download by `torch.hub.list`.

In [20]:
torch.hub.list('pytorch/vision')

Using cache found in /root/.cache/torch/hub/pytorch_vision_master


['alexnet',
 'deeplabv3_resnet101',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'fcn_resnet101',
 'googlenet',
 'inception_v3',
 'mobilenet_v2',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn',
 'wide_resnet101_2',
 'wide_resnet50_2']